In [ ]:
import sys
import os
import pandas as pd
import warnings
import numpy as np
from datetime import date
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
sys.path.append(os.path.abspath("/projects/genomic-ml/da2343/ml_project_1/shared"))
from model_header import *
from constants import *

In [ ]:
warnings.filterwarnings("ignore")
np.set_printoptions(threshold=np.inf)
params_df = pd.read_csv("params.csv")
param_row = 0
param_dict = dict(params_df.iloc[param_row, :])
data_set_name = param_dict["Dataset"]
n_samples = param_dict["# of Total Samples"]
index_of_pred_col = param_dict["Index of Prediction Col"]

dataset_path = dataset_dict[data_set_name]
n_splits = 3
# Import the csv file of the dataset
df = pd.read_csv(dataset_path, header=0)
learner_dict = {
    "Featureless": Featureless(),
    "LassoCV": LassoCV(random_state=1),
}

test_err_list = []
pred_col_name = df.columns[index_of_pred_col]
output_vec = df.iloc[:, index_of_pred_col].to_numpy().ravel()
input_mat = df.drop(pred_col_name, axis=1).to_numpy()

In [ ]:
output_vec.shape

In [ ]:
k_fold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
for fold_id, indices in enumerate(k_fold.split(input_mat)):
    index_dict = dict(zip(["train", "test"], indices))
    set_data_dict = {}
    for set_name, index_vec in index_dict.items():
        set_data_dict[set_name] = {
            "X": input_mat[index_vec],
            "y": output_vec[index_vec],
        }
    # Fit the learner to the training data
    # Predict the test data
    # Calculate the test error
    for learner_name, learner in learner_dict.items():
        learner.fit(**set_data_dict["train"])
        pred_y = learner.predict(set_data_dict["test"]["X"])
        actual_y = set_data_dict["test"]["y"]
        mse = mean_squared_error(actual_y, pred_y)
        test_err_list.append(
            pd.DataFrame(
                {
                    "Mean Squared Error": mse,
                    "FoldID": fold_id,
                    "# of Total Samples": n_samples,
                    "Dataset": data_set_name,
                    "Index of Predicted Column": index_of_pred_col,
                    "pred_col_name": pred_col_name,
                    "Algorithm": learner_name
                },
                index=[0],
            )
        )
test_err_df = pd.concat(test_err_list)

In [ ]:
# Save dataframe as a csv to output directory
out_file = f"results/{param_row}.csv"
test_err_df.to_csv(out_file, encoding="utf-8", index=False)
print("Done!!")

In [16]:
import sys
import os
import pandas as pd
import warnings
import numpy as np
from datetime import date
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import *
sys.path.append(os.path.abspath("/projects/genomic-ml/da2343/ml_project_1/shared"))
from model_header import *
from constants import *
from sklearn.dummy import *


warnings.filterwarnings("ignore")
np.set_printoptions(threshold=np.inf)
params_df = pd.read_csv("params.csv")
param_row = 0
param_dict = dict(params_df.iloc[param_row, :])
data_set_name = param_dict["Dataset"]
index_of_pred_col = param_dict["Index of Prediction Col"]

dataset_path = dataset_dict[data_set_name]
n_splits = 3
# Import the csv file of the dataset
df = pd.read_csv(dataset_path, header=0)
reg_learner_dict = {
    "Featureless": Featureless(),
    "LassoCV": LassoCV(random_state=1),
}
classifier_dict = {
    "FeaturelessClassifier": DummyClassifier(strategy="most_frequent"),
    "LogisticRegression": LogisticRegressionCV(random_state=1),
}

test_err_list = []
pred_col_name = df.columns[index_of_pred_col]

# two output vectors
# one will just be the output of the regression
output_vec_for_reg = df.iloc[:, index_of_pred_col].to_numpy().ravel()
# the other will be the output of the binary classifier
# when the elements in output_vec_for_reg is greater than 0 then the corresponding element in output_vec_classifier will be 1 otherwise it will be 0
output_vec_for_class = np.where(output_vec_for_reg > 0, 1, 0)
input_mat = df.drop(pred_col_name, axis=1).to_numpy()


k_fold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
for fold_id, indices in enumerate(k_fold.split(input_mat)):
    index_dict = dict(zip(["train", "test"], indices))
    set_data_dict = {}
    for set_name, index_vec in index_dict.items():
        set_data_dict[set_name] = {
            "X": input_mat[index_vec],
            "y": output_vec_for_class[index_vec],
        }
    # Fit the learner to the training data
    # Predict the test data
    # Calculate the test error
    for learner_name, learner in classifier_dict.items():
        learner.fit(**set_data_dict["train"])
        pred_y = learner.predict(set_data_dict["test"]["X"])
        actual_y = set_data_dict["test"]["y"]
        accuracy = np.mean(pred_y == actual_y)
        
        fpr, tpr, _ = roc_curve(actual_y, pred_y)
        
        test_err_list.append(
            pd.DataFrame(
                {
                    "Test Accuracy": accuracy,
                    "FoldID": fold_id,
                    "Dataset": data_set_name,
                    "Index of Predicted Column": pred_col_name,
                    "Algorithm": learner_name,
                    "AUC": roc_auc_score(actual_y, pred_y),
                    "FPR": np.array2string(fpr),
                    "TPR": np.array2string(tpr),
                },
                index=[0],
            )
        )
test_err_df = pd.concat(test_err_list)

In [17]:
test_err_df

,Test Accuracy,FoldID,Dataset,Index of Predicted Column,Algorithm,AUC,FPR,TPR
0,0.478261,0,Dec22_all_power,Bacillus,FeaturelessClassifier,0.500000,[0. 1.],[0. 1.]
0,0.782609,0,Dec22_all_power,Bacillus,LogisticRegression,0.784091,[0. 0.18181818 1. ],[0. 0.75 1. ]
0,0.434783,1,Dec22_all_power,Bacillus,FeaturelessClassifier,0.500000,[0. 1.],[0. 1.]
0,0.652174,1,Dec22_all_power,Bacillus,LogisticRegression,0.634615,[0. 0.23076923 1. ],[0. 0.5 1. ]
0,0.478261,2,Dec22_all_power,Bacillus,FeaturelessClassifier,0.500000,[0. 1.],[0. 1.]
0,0.739130,2,Dec22_all_power,Bacillus,LogisticRegression,0.742424,[0. 0.18181818 1. ],[0. 0.66666667 1. ]


In [14]:
fpr_string = np.array2string(fpr)
fpr_string

'[0.         0.18181818 1.        ]'

In [15]:
# string to numpy array
fpr = np.fromstring(fpr_string[1:-1], sep=" ")
fpr

array([0.        , 0.18181818, 1.        ])

In [10]:
# convert the fpr and tpr to a string
type(fpr)

numpy.ndarray

In [ ]:
output = roc_curve(actual_y, pred_y)
type(output)

In [ ]:
import sys
import os
import pandas as pd
import warnings
import numpy as np
from datetime import date
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import *
sys.path.append(os.path.abspath("/projects/genomic-ml/da2343/ml_project_1/shared"))
from model_header import *
from constants import *
from sklearn.dummy import *

warnings.filterwarnings("ignore")
np.set_printoptions(threshold=np.inf)
params_df = pd.read_csv("params.csv")

# if len(sys.argv) == 2:
#     prog_name, task_str = sys.argv
#     param_row = int(task_str)
# else:
#     print("len(sys.argv)=%d so trying first param" % len(sys.argv))
#     param_row = 0

param_row = 0
param_dict = dict(params_df.iloc[param_row, :])
data_set_name = param_dict["Dataset"]
index_of_pred_col = param_dict["Index of Prediction Col"]

dataset_path = dataset_dict[data_set_name]
n_splits = 3
# Import the csv file of the dataset
df = pd.read_csv(dataset_path, header=0)

classifier_reg_dict = {
    "Featureless": {
        "classifier": DummyClassifier(strategy="most_frequent"),
        "regressor": Featureless(),
     },
    "LogisticRegLassoCV": {
        "classifier": LogisticRegressionCV(random_state=1),
        "regressor": LassoCV(random_state=1),
    }
}    
    

test_err_list = []
pred_col_name = df.columns[index_of_pred_col]

# two output vectors
# one will just be the output of the regression
output_vec_for_reg = df.iloc[:, index_of_pred_col].to_numpy().ravel()
output_vec_for_class = np.where(output_vec_for_reg > 0, 1, 0)
input_mat = df.drop(pred_col_name, axis=1).to_numpy()


k_fold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
for fold_id, indices in enumerate(k_fold.split(input_mat)):
    index_dict = dict(zip(["train", "test"], indices))
    set_data_dict = {}
    for set_name, index_vec in index_dict.items():
        set_data_dict[set_name] = {
            "X": input_mat[index_vec],
            "y_class": output_vec_for_class[index_vec],
            "y_reg": output_vec_for_reg[index_vec],
        }
    # Fit the learner to the training data
    # Predict the test data
    # Calculate the test error
    for learner_name, learner in classifier_reg_dict.items():
        # check if y_train contains only one class
        if len(np.unique(set_data_dict["train"]["y_class"])) == 1:
            # predict all test data as the class that is present in y_train
            pred_y = np.full(set_data_dict["test"]["y_class"].shape, 
                             np.unique(set_data_dict["train"]["y_class"])[0])
        else:
            classifier = learner["classifier"]
            classifier.fit(set_data_dict["train"]["X"], set_data_dict["train"]["y_class"])
            classifier_pred_y = classifier.predict(set_data_dict["test"]["X"])
            
            regressor = learner["regressor"]
            regressor.fit(set_data_dict["train"]["X"], set_data_dict["train"]["y_reg"])
            regressor_pred_y = regressor.predict(set_data_dict["test"]["X"])
            # when the classifier predicts 0 then the regressor prediction is set to 0
            # when the classifier predicts 1 then the regressor prediction is set to the regressor prediction
            pred_y = np.where(classifier_pred_y == 0, 0, regressor_pred_y)
            
        actual_y = set_data_dict["test"]["y_reg"]
        test_err_list.append(
            pd.DataFrame(
                {
                    "Mean Squared Error": mean_squared_error(actual_y, pred_y),
                    "FoldID": fold_id,
                    "Dataset": data_set_name,
                    "Index of Predicted Column": index_of_pred_col,
                    "Predicted Column Name": pred_col_name,
                    "Algorithm": learner_name
                },
                index=[0],
            )
        )
test_err_df = pd.concat(test_err_list)
# Save dataframe as a csv to output directory
# out_file = f"results/{param_row}.csv"
# test_err_df.to_csv(out_file, encoding="utf-8", index=False)
# print("Done!!")

In [ ]:
test_err_df

In [ ]:
df